In [1]:
import pandas as pd  
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_sizes, output_size, learning_rate):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        self.weights = [np.random.rand(input_size, hidden_sizes[0])]
        self.biases = [np.zeros((1, hidden_sizes[0]))]
        
        for i in range(1, len(hidden_sizes)):
            self.weights.append(np.random.rand(hidden_sizes[i-1], hidden_sizes[i]))
            self.biases.append(np.zeros((1, hidden_sizes[i])))
        
        self.weights.append(np.random.rand(hidden_sizes[-1], output_size))
        self.biases.append(np.zeros((1, output_size)))
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward(self, x):
        self.layer_outputs = [x]
        
        for i in range(len(self.weights)):
            layer_input = np.dot(self.layer_outputs[i], self.weights[i]) + self.biases[i]
            layer_output = self.sigmoid(layer_input)
            self.layer_outputs.append(layer_output)
        
        return self.layer_outputs[-1]
        
    def backward(self, x, y, output):
        self.output_error = y - output
        self.output_delta = self.output_error * self.sigmoid_derivative(output)

        for i in range(len(self.weights) - 1, -1, -1):
            self.weights[i] += self.learning_rate * np.dot(self.layer_outputs[i].T, self.output_delta)
            self.biases[i] += self.learning_rate * np.sum(self.output_delta, axis=0, keepdims=True)

            hidden_error = np.dot(self.output_delta, self.weights[i].T)
            hidden_delta = hidden_error * self.sigmoid_derivative(self.layer_outputs[i])

            self.output_delta = hidden_delta
    
    def train(self, x, y, epochs, batch_size):
        for epoch in range(epochs):
            for batch_start in range(0, len(x), batch_size):
                batch_end = batch_start + batch_size
                batch_x = x[batch_start:batch_end]
                batch_y = y[batch_start:batch_end]

                output = self.forward(batch_x)
                self.backward(batch_x, batch_y, output)

            batch_predictions = self.predict(batch_x)
            loss = np.mean(np.square(batch_y - batch_predictions))
            if epoch % 20 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")
    
    def predict(self, x):
        return self.forward(x)

In [3]:
heart = pd.read_csv("heart.csv")
df = heart.copy()

In [4]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [6]:
df['target'].value_counts()

1    165
0    138
Name: target, dtype: int64

In [7]:
df.cp.unique()

array([3, 2, 1, 0], dtype=int64)

In [8]:
df.thal.unique()

array([1, 2, 3, 0], dtype=int64)

In [9]:
df[['cp', 'thal']] = df[['cp', 'thal']].astype(object)
objects = df[['cp', 'thal']]
objects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cp      303 non-null    object
 1   thal    303 non-null    object
dtypes: object(2)
memory usage: 4.9+ KB


In [10]:
objects = pd.get_dummies(objects, drop_first = True)
df_ = df.drop(df.select_dtypes(include=['object']).columns, axis = 1)
df = pd.concat([df_, objects], axis = 1)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   trestbps  303 non-null    int64  
 3   chol      303 non-null    int64  
 4   fbs       303 non-null    int64  
 5   restecg   303 non-null    int64  
 6   thalach   303 non-null    int64  
 7   exang     303 non-null    int64  
 8   oldpeak   303 non-null    float64
 9   slope     303 non-null    int64  
 10  ca        303 non-null    int64  
 11  target    303 non-null    int64  
 12  cp_1      303 non-null    uint8  
 13  cp_2      303 non-null    uint8  
 14  cp_3      303 non-null    uint8  
 15  thal_1    303 non-null    uint8  
 16  thal_2    303 non-null    uint8  
 17  thal_3    303 non-null    uint8  
dtypes: float64(1), int64(11), uint8(6)
memory usage: 30.3 KB


In [12]:
y = df.target
X = df.drop('target', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.24, random_state=111, shuffle=True)

In [13]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((230, 17), (73, 17), (230,), (73,))

In [14]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
y_train = np.array(y_train, dtype=np.int64).reshape(len(y_train),1)
y_test = np.array(y_test, dtype=np.int64).reshape(len(y_test),1)

In [16]:
learning_rate = 0.03
input_size = X_train.shape[1]
hidden_sizes = [6, 4]
output_size = 1

In [17]:
nn = NeuralNetwork(input_size, hidden_sizes, output_size, learning_rate)

In [18]:
epochs = 200
batch_size = 50

In [19]:
nn.train(X_train, y_train, epochs, batch_size)

Epoch 0, Loss: 0.2854
Epoch 20, Loss: 0.2494
Epoch 40, Loss: 0.2480
Epoch 60, Loss: 0.2393
Epoch 80, Loss: 0.1807
Epoch 100, Loss: 0.1584
Epoch 120, Loss: 0.1535
Epoch 140, Loss: 0.1501
Epoch 160, Loss: 0.1477
Epoch 180, Loss: 0.1457


In [20]:
y_pred = np.round(nn.predict(X_test))

In [21]:
accuracy_score(y_test, y_pred)

0.8767123287671232

In [22]:
print(confusion_matrix(y_test, y_pred))

[[22  6]
 [ 3 42]]


In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.79      0.83        28
           1       0.88      0.93      0.90        45

    accuracy                           0.88        73
   macro avg       0.88      0.86      0.87        73
weighted avg       0.88      0.88      0.88        73

